# Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---

## Import Packages

In [1]:
# Importing some useful packages
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from helper_functions import *
import pickle

#Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib qt


## 1 - Compute the camera calibration using chessboard images

In [2]:
# define the name of the output directory to be created
output_path = "output_images/camera_calibration"

# Create test_images_output Directory if doesn't exist
if not os.path.exists(output_path):
    os.mkdir(output_path)

In [3]:
# To avoid running the calibration step everytime, in case the camera coefficients already exist, just load it
if os.path.exists('camera_coeff.pkl'):
    # Getting back the values:
    with open('camera_coeff.pkl', 'rb') as f:  
        mtx, dist = pickle.load(f)
    
else:  
    # Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(9,6,0)
    objp = np.zeros((9*6,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob('camera_cal/calibration*.jpg')

    # Step through the list and search for chessboard corners
    for fname in images:

        # Read the image
        img = cv2.imread(fname)

        # Convert the image to gray scale
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Find the chessboard 54 (9*6) corners 
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

        # If found, append object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
            #cv2.imshow('img',img)
            #cv2.waitKey(500)

    #cv2.destroyAllWindows()

    # Perform the camera calibration
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None) 

    # Store the camera calibration coefficients for future use
    with open('camera_coeff.pkl', 'wb') as f:  
        pickle.dump([mtx, dist], f)
        
    # Perform distortion correction on chessboard images to verify the process is doing well

    # Step through the list and undistort each image
    for fname in images:

        # Read the image
        image = mpimg.imread(fname)

        # Undistort the image an display it 
        undist = cv2.undistort(image, mtx, dist, None, mtx)

        # Save images on output_path Directory
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
        f.tight_layout()

        ax1.imshow(image)
        ax1.set_title('Original Image', fontsize=30)

        ax2.imshow(undist)
        ax2.set_title('Undistorted Image', fontsize=30)
        plt.subplots_adjust(left=0.1, right=0.9, top=1, bottom=0, wspace = 0.1)


        plt.savefig(os.path.join(output_path, "undist_" + os.path.basename(fname)))
        plt.close()

## 2 - Build a Lane finding Pipeline
The pipeline will be composed by the following steps:

1 - Read and apply a distortion correction to raw images

2 - Use color transforms, gradients, etc., to create a thresholded binary image

3 - Apply a perspective transform to rectify binary image ("birds-eye view")

4 - Detect lane pixels and fit to find the lane boundary

5 - Determine the curvature of the lane and vehicle position with respect to center 

6 - Warp the detected lane boundaries back onto the original image

7 - Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position

In [4]:
# define the name of the directory to be created
output_path = "output_images/test_images"

# Create test_images_output Directory if doesn't exist
if not os.path.exists(output_path):
    os.mkdir(output_path)

In [5]:
# Get the transform matrix using 4 source and destination points calculated manually looking to 
# the straight line images

src = np.float32([[195, 720],[1125, 720],[578, 460],[705, 460]])
dst = np.float32([[350, 720],[950, 720],[350,0],[950,0]])

# Get the transform matrix M using the src and dst points
M = cv2.getPerspectiveTransform(src, dst)

# Get the invert transform matrix M using the src and dst points
M_inv = cv2.getPerspectiveTransform(dst, src)
        

In [6]:
def process_image(original_image, fname, mtx, dist, M, M_inv):
    
    ## 1 - Apply a distortion correction to the image ##
    undist = undistort_image(original_image, mtx, dist)
    save_undistorted_images(output_path, fname, original_image, undist)
    

    ## 2 - Use color transforms, gradients, etc., to create a thresholded binary image ##
    ksize = 3 # Sobel kernel size 

    # Apply each of the gradient thresholding functions
    gradx = abs_sobel_thresh(undist, orient='x', sobel_kernel=ksize, thresh=(30, 200))
    grady = abs_sobel_thresh(undist, orient='y', sobel_kernel=ksize, thresh=(50, 200))
    mag_binary = mag_thresh(undist, sobel_kernel=ksize, mag_thresh=(30, 200))
    dir_binary = dir_threshold(undist, sobel_kernel=ksize, thresh=(0.7, 1.3))

    # Apply each of the color thresholding functions
    colors_binary = hls_select(undist, s_thresh=(200, 255), v_thresh=(230, 255))

    # Combine all of the thresholding binaries
    binary_image = np.zeros_like(gradx)
    #binary_image[(colors_binary == 1) | ((mag_binary == 1) & (dir_binary == 1)) | ((gradx == 1) & (grady == 1)) ] = 1
    binary_image[(colors_binary == 1) | (gradx == 1) ] = 1
   
    save_binary_images(output_path, fname, undist, binary_image)
    
    # 3 - Apply a perspective transform to rectify binary image ("birds-eye view") ##

    # Warp the image to a top-down view
    img_size = (undist.shape[1], undist.shape[0])
    binary_warped = cv2.warpPerspective(binary_image, M, img_size, flags=cv2.INTER_LINEAR)
    
    warped = cv2.warpPerspective(undist, M, img_size, flags=cv2.INTER_LINEAR)
    save_warped_images(output_path, fname, binary_warped, warped)

    ## 4 - Detect lane pixels and fit to find the lane boundary ##
    
    # Create a sliding window and find out which activated pixels fall into the window
    out_img, left_fit, right_fit, left_fitx, right_fitx, ploty = fit_polynomial(binary_warped)

    ## 5 - Determine the curvature of the lane and vehicle position with respect to center ##

    radius_curvature = measure_curvature_real(out_img.shape[0], left_fit, right_fit)

    rel_vehicle_position = measure_rel_vehicle_position(out_img.shape, left_fit, right_fit)

    ## 6 - Warp the detected lane boundaries back onto the original image ##

    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, M_inv, img_size) 

    ## 7 - Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position ##
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    result = cv2.addWeighted(original_image, 1, newwarp, 0.3, 0)

    save_lane_lines_image(output_path, fname, out_img)

    print("Radius curvature = ", radius_curvature, 'm')
    print("Relative vehicle position with respect to the line lane center = ",rel_vehicle_position, 'm')

    save_pipeline_image(output_path, fname, result, radius_curvature, rel_vehicle_position )

In [7]:
%matplotlib inline

# Make a list of test images
images = glob.glob('test_images/*.jpg')
images = []

# Step through the list, read the image and apply the lane finding pipeline
for fname in images:

    print("-----------------------", fname, "-----------------------")

    # Read the image
    img = mpimg.imread(fname)
    img_size = img.shape

    process_image(img, fname, mtx, dist, M, M_inv)

    print('\n')

## 3 -  Test the pipeline on Videos

In [8]:
from frame import *
img_size = (720,1280)

In [9]:
white_output = 'project_video_out.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(Frame(mtx, dist, M, M_inv, img_size)).subclip(0,5)
%time white_clip.write_videofile(white_output, audio=False)


t:   2%|▏         | 2/125 [00:00<00:09, 13.04it/s, now=None]

Moviepy - Building video project_video_out.mp4.
Moviepy - Writing video project_video_out.mp4



t:  13%|█▎        | 16/125 [00:02<00:16,  6.58it/s, now=None]

error 1 1587.3029196524133 713.6051721983468
error 1 1756.4676566173441 736.964818251616


t:  14%|█▍        | 18/125 [00:02<00:16,  6.62it/s, now=None]

...

t:  68%|██████▊   | 85/125 [00:13<00:06,  6.21it/s, now=None]

error 1 748.1739887704514 1760.6414797102539
error 1 770.38439960031 1815.8258879985056


t:  70%|███████   | 88/125 [00:13<00:06,  5.83it/s, now=None]

error 1 833.3702801255521 1808.5299307515306
error 1 838.1777088010369 1696.4852153318702


Moviepy - Done !
Moviepy - video ready project_video_out.mp4
CPU times: user 47.3 s, sys: 8.64 s, total: 55.9 s
Wall time: 21.7 s


In [10]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## 4 - Test the pipeline on Challenge Videos

In [11]:
# Get the transform matrix using 4 source and destination points calculated looking to the straight line image
src = np.float32([[250, 720],[1050, 720],[605, 480],[715, 480]])
dst = np.float32([[350, 720],[950, 720],[350,0],[950,0]])

# Get the transform matrix M using the src and dst points
M_1 = cv2.getPerspectiveTransform(src, dst)

# Get the invert transform matrix M using the src and dst points
M_inv_1 = cv2.getPerspectiveTransform(dst, src)

fname = "challenge_video.jpg"

# Read the image
clip1 = VideoFileClip("challenge_video.mp4")
img = clip1.get_frame(130 / clip1.fps) # get frame by index
img_size = img.shape

process_image(img, fname, mtx, dist, M, M_inv)

print('\n')

Radius curvature =  332 m
Relative vehicle position with respect to the line lane center =  0.11 m




In [12]:
white_output = 'challenge_video_out.mp4'
clip1 = VideoFileClip("challenge_video.mp4")
white_clip = clip1.fl_image(Frame(mtx, dist, M, M_inv, img_size)).subclip(0,5) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

t:   0%|          | 0/150 [00:00<?, ?it/s, now=None]

error 1 1824.7861540674053 222.40493662060302
error 1 805.5526014888259 227.56759428773228
Moviepy - Building video challenge_video_out.mp4.
Moviepy - Writing video challenge_video_out.mp4



t:   2%|▏         | 3/150 [00:00<00:14, 10.08it/s, now=None]

error 1 699.9639782101178 229.34372485003487


t:  11%|█         | 16/150 [00:02<00:19,  7.00it/s, now=None]

...

t:  99%|█████████▊| 148/150 [00:21<00:00,  6.01it/s, now=None]

error 1 303.4710157940267 3.5534246901079376
error 4 -19828.258737710083 1015.6759159564792 20843.93465366656


t: 100%|██████████| 150/150 [00:22<00:00,  5.84it/s, now=None]

error 1 301.9513644541697 3.9135030015355174
error 4 -17697.335705678976 1033.2286442226728 18730.564349901648
error 1 328.7563681901047 5.982219232100929
error 4 -11202.544377399036 1044.7652205161282 12247.309597915164


error 1 330.3692907375821 1867.5766922760433
Moviepy - Done !
Moviepy - video ready challenge_video_out.mp4
CPU times: user 25.9 s, sys: 4.48 s, total: 30.4 s
Wall time: 24.9 s


In [13]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [ ]:
white_output = 'harder_challenge_video_out.mp4'
clip1 = VideoFileClip("harder_challenge_video.mp4")
white_clip = clip1.fl_image(Frame(mtx, dist, M, M_inv, img_size)).subclip(0,5) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

t:   2%|▏         | 2/125 [00:00<00:09, 12.38it/s, now=None]

Moviepy - Building video harder_challenge_video_out.mp4.
Moviepy - Writing video harder_challenge_video_out.mp4



t:  16%|█▌        | 20/125 [00:03<00:16,  6.31it/s, now=None]

error 1 707.8997264882998 273.2381523659142
error 1 846.9717362294252 263.8908707504843


t:  18%|█▊        | 22/125 [00:03<00:16,  6.33it/s, now=None]

...

error 1 47.611659428862 188.5017430244318
error 4 1342.4490209120972 1619.6829907493116 277.23396983721443


t:  99%|█████████▉| 124/125 [00:23<00:00,  3.98it/s, now=None]

error 1 67.8340913227055 200.86621578845435
error 4 900.2905242890745 1171.940353539426 271.64982925035156


t: 100%|██████████| 125/125 [00:24<00:00,  3.67it/s, now=None]

error 1 77.96175906097122 187.12466661134246


In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))